In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install textstat
!pip install lexicalrichness

     |████████████████████████████████| 101 kB 3.4 MB/s 
     |████████████████████████████████| 2.0 MB 47.0 MB/s 
  Created wheel for lexicalrichness: filename=lexicalrichness-0.1.4-py2.py3-none-any.whl size=10109 sha256=57d8e48c2ba5c593c78481a0c24c682e2a292db3c0b4a314f3f181878654789f
  Stored in directory: /root/.cache/pip/wheels/40/53/09/ce0a119b59493ae5be4e9773457df832bbce66d926fce1d043
Successfully built lexicalrichness


In [ ]:
#import các thư viện 
import nltk
from nltk.corpus import stopwords  
nltk.download("stopwords") 
stop_words = set(stopwords.words('english'))

from nltk.tag import pos_tag
from nltk import word_tokenize
from collections import Counter

import textstat
from lexicalrichness import LexicalRichness

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

#load dataset
pd.set_option('display.max_columns', 500)
df = pd.read_csv("https://raw.githubusercontent.com/ltdaovn/dataset/master/corona_fake.csv")

#cập nhật lại các giá trị của các thuộc tính và loại bỏ giá trị nan
df.loc[df['label'] == 'Fake', ['label']] = 'FAKE'
df.loc[df['label'] == 'fake', ['label']] = 'FAKE'
df.loc[df['source'] == 'facebook', ['source']] = 'Facebook'
df.text.fillna(df.title, inplace=True)

df.loc[5]['label'] = 'FAKE'
df.loc[15]['label'] = 'TRUE'
df.loc[43]['label'] = 'FAKE'
df.loc[131]['label'] = 'TRUE'
df.loc[242]['label'] = 'FAKE'

df = df.sample(frac=1).reset_index(drop=True)
df.title.fillna('missing', inplace=True)
df.source.fillna('missing', inplace=True)
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,text,source,label
0,What are the symptoms of COVID-19?,How will you know if you have the novel corona...,https://www.livescience.com/,TRUE
1,missing,Coronavirus does not cause a runny nose; is ki...,Change Your Thoughts Change Your Life,FAKE
2,What is Coronavirus disease 2019?,"Coronavirus disease 2019, or COVID-19, is a ne...",https://www.chop.edu/,TRUE
3,missing,Breathing deeply for just a few minutes in any...,YouTube,FAKE
4,missing,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,FAKE
...,...,...,...,...
1159,WATCH: BILL GATES LETS YOU KNOW THAT MASS GATH...,"In October of 2012, this website ran a story a...",https://www.wakingtimes.com/,FAKE
1160,RaTG13 – the Undeniable Evidence That the Wuha...,What is the true origin of the Wuhan coronavir...,https://gnews.org/,FAKE
1161,missing,The European Union is actually dead. In this c...,Youtube,FAKE
1162,There is no link between the coronavirus and 5...,The EU has the highest consumer standards in t...,https://ec.europa.eu/,TRUE


In [ ]:
df['title_num_uppercase'] = df['title'].str.count(r'[A-Z]')
df['text_num_uppercase'] = df['text'].str.count(r'[A-Z]')
df['text_len'] = df['text'].str.len()
df['text_pct_uppercase'] = df.text_num_uppercase.div(df.text_len)

x1 = df.loc[df['label']=='TRUE']['title_num_uppercase']
x2 = df.loc[df['label'] == 'FAKE']['title_num_uppercase']

group_labels = ['TRUE', 'FAKE']
colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Distribution of Uppercase in title', template="plotly_white")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=x1, name='TRUE',
                marker_color = 'rgb(0, 0, 100)'))
fig.add_trace(go.Box(y=x2, name = 'FAKE',
                marker_color = 'rgb(0, 200, 200)'))
fig.update_layout(title_text='Box plot of Capital Letter in title', template="plotly_white")
fig.show()

In [ ]:
df['title_num_stop_words'] = df['title'].str.split().apply(lambda x: len(set(x) & stop_words))
df['text_num_stop_words'] = df['text'].str.split().apply(lambda x: len(set(x) & stop_words))
df['text_word_count'] = df['text'].apply(lambda x: len(str(x).split()))
df['text_pct_stop_words'] = df['text_num_stop_words'] / df['text_word_count']

x1 = df.loc[df['label']=='TRUE']['title_num_stop_words']
x2 = df.loc[df['label'] == 'FAKE']['title_num_stop_words']
group_labels = ['TRUE', 'FAKE']
colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Distribution of Stop Words in title', template="plotly_white")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=x1, name='TRUE',
                marker_color = 'rgb(0, 0, 100)'))
fig.add_trace(go.Box(y=x2, name = 'FAKE',
                marker_color = 'rgb(0, 200, 200)'))
fig.update_layout(title_text='Box plot of Stop Words in title', template="plotly_white")
fig.show()

In [ ]:
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
df.drop(['text_num_uppercase', 'text_len', 'text_num_stop_words', 'text_word_count'], axis=1, inplace=True)

df['token'] = df.apply(lambda row: word_tokenize(row['title']), axis=1)
df['pos_tags'] = df.apply(lambda row: pos_tag(row['token']), axis=1)
tag_count_df = pd.DataFrame(df['pos_tags'].map(lambda x: Counter(tag[1] for tag in x)).to_list())
df = pd.concat([df, tag_count_df], axis=1).fillna(0).drop(['pos_tags', 'token'], axis=1)
df = df[['title', 'text', 'source', 'label', 'title_num_uppercase', 'text_pct_uppercase', 'title_num_stop_words', 'text_pct_stop_words', 'NNP']].rename(columns={'NNP': 'NNP_title'})
x1 = df.loc[df['label']=='TRUE']['NNP_title']
x2 = df.loc[df['label'] == 'FAKE']['NNP_title']

group_labels = ['TRUE', 'FAKE']

colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Number of Proper nouns in title', template="plotly_white")
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=x1, name='TRUE',
                marker_color = 'rgb(0, 0, 100)'))
fig.add_trace(go.Box(y=x2, name = 'FAKE',
                marker_color = 'rgb(0, 200, 200)'))
fig.update_layout(title_text='Box plot of Proper nouns in title', template="plotly_white")
fig.show()

In [ ]:
df['token'] = df.apply(lambda row: word_tokenize(row['text']), axis=1)
df['pos_tags'] = df.apply(lambda row: pos_tag(row['token']), axis=1)

tag_count_df = pd.DataFrame(df['pos_tags'].map(lambda x: Counter(tag[1] for tag in x)).to_list())

df = pd.concat([df, tag_count_df], axis=1).fillna(0).drop(['pos_tags', 'token'], axis=1)

In [ ]:
df['num_negation'] = df['text'].str.lower().str.count("no|not|never|none|nothing|nobody|neither|nowhere|hardly|scarcely|barely|doesn’t|isn’t|wasn’t|shouldn’t|wouldn’t|couldn’t|won’t|can't|don't")

df['num_interrogatives_title'] = df['title'].str.lower().str.count("what|who|when|where|which|why|how")
df['num_interrogatives_text'] = df['text'].str.lower().str.count("what|who|when|where|which|why|how")


In [ ]:
reading_ease = []
for doc in df['text']:
    reading_ease.append(textstat.flesch_reading_ease(doc))
    
smog = []
for doc in df['text']:
    smog.append(textstat.smog_index(doc))
    
kincaid_grade = []
for doc in df['text']:
    kincaid_grade.append(textstat.flesch_kincaid_grade(doc))
    
liau_index = []
for doc in df['text']:
    liau_index.append(textstat.coleman_liau_index(doc))
    
readability_index = []
for doc in df['text']:
    readability_index.append(textstat.automated_readability_index(doc))

readability_score = []
for doc in df['text']:
    readability_score.append(textstat.dale_chall_readability_score(doc))
    
difficult_words = []
for doc in df['text']:
    difficult_words.append(textstat.difficult_words(doc))

write_formula = []
for doc in df['text']:
    write_formula.append(textstat.linsear_write_formula(doc))

gunning_fog = []
for doc in df['text']:
    gunning_fog.append(textstat.gunning_fog(doc))

text_standard = []
for doc in df['text']:
    text_standard.append(textstat.text_standard(doc))
    
df['flesch_reading_ease'] = reading_ease
df['smog_index'] = smog
df['flesch_kincaid_grade'] = kincaid_grade
df['automated_readability_index'] = readability_index
df['dale_chall_readability_score'] = readability_score
df['difficult_words'] = difficult_words
df['linsear_write_formula'] = write_formula
df['gunning_fog'] = gunning_fog
df['text_standard'] = text_standard


In [ ]:
ttr = []
for doc in df['text']:
    lex = LexicalRichness(doc)
    ttr.append(lex.ttr)

df['ttr'] = ttr


df['num_powerWords_text'] = df['text'].str.lower().str.count('improve|trust|immediately|discover|profit|learn|know|understand|powerful|best|win|more|bonus|exclusive|extra|you|free|health|guarantee|new|proven|safety|money|now|today|results|protect|help|easy|amazing|latest|extraordinary|how to|worst|ultimate|hot|first|big|anniversary|premiere|basic|complete|save|plus|create')
df['num_casualWords_text'] = df['text'].str.lower().str.count('make|because|how|why|change|use|since|reason|therefore|result')
df['num_tentativeWords_text'] = df['text'].str.lower().str.count('may|might|can|could|possibly|probably|it is likely|it is unlikely|it is possible|it is probable|tends to|appears to|suggests that|seems to')
df['num_emotionWords_text'] = df['text'].str.lower().str.count('ordeal|outrageous|provoke|repulsive|scandal|severe|shameful|shocking|terrible|tragic|unreliable|unstable|wicked|aggravate|agony|appalled|atrocious|corruption|damage|disastrous|disgusted|dreadful|eliminate|harmful|harsh|inconsiderate|enraged|offensive|aggressive|frustrated|controlling|resentful|anger|sad|fear|malicious|infuriated|critical|violent|vindictive|furious|contrary|condemning|sarcastic|poisonous|jealous|retaliating|desperate|alienated|unjustified|violated')


In [ ]:
x1 = df.loc[df['label']=='TRUE']['text_pct_uppercase']
x2 = df.loc[df['label'] == 'FAKE']['text_pct_uppercase']

group_labels = ['TRUE', 'FAKE']

colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Percentage of Capital Letter in Article body', template="plotly_white")
fig.show()

In [ ]:
x1 = df.loc[df['label']=='TRUE']['text_pct_stop_words']
x2 = df.loc[df['label'] == 'FAKE']['text_pct_stop_words']

group_labels = ['TRUE', 'FAKE']

colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Percentage of Stop Words in Text', template="plotly_white")
fig.show()

In [ ]:

x1 = df.loc[df['label']=='TRUE']['NNP']
x2 = df.loc[df['label'] == 'FAKE']['NNP']

group_labels = ['TRUE', 'FAKE']

colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Number of Proper noun in Article Body', template="plotly_white")
fig.show()

In [ ]:
x1 = df.loc[df['label']=='TRUE']['num_negation']
x2 = df.loc[df['label'] == 'FAKE']['num_negation']

group_labels = ['TRUE', 'FAKE']

colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Number of Negations in Text', template="plotly_white")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=x1, name='TRUE',
                marker_color = 'rgb(0, 0, 100)'))
fig.add_trace(go.Box(y=x2, name = 'FAKE',
                marker_color = 'rgb(0, 200, 200)'))
fig.update_layout(title_text='Box plot of Brackets in Article Bodies', template="plotly_white")
fig.show()

In [ ]:
x1 = df.loc[df['label']=='TRUE']['ttr']
x2 = df.loc[df['label'] == 'FAKE']['ttr']

group_labels = ['TRUE', 'FAKE']

colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Type-token ratio in Article Bodies', template="plotly_white")
fig.show()

In [ ]:
x1 = df.loc[df['source']=='https://www.health.harvard.edu/']['text_pct_stop_words']
x2 = df.loc[df['source']=='https://www.naturalnews.com/']['text_pct_stop_words']

group_labels = ['Health Harvard', 'Natural News']

colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Percentage of Stop Words in Article Bodies', template="plotly_white")
fig.show()

In [ ]:
x1 = df.loc[df['source']=='https://www.health.harvard.edu/']['ttr']
x2 = df.loc[df['source'] == 'https://www.naturalnews.com/']['ttr']

group_labels = ['Harvard', 'Natural News']

colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

fig = ff.create_distplot(
    [x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='Type-token ratio in Article Bodies', template="plotly_white")
fig.show()

In [ ]:
X, y = df.drop(['title', 'text', 'source', 'label', 'text_standard'], axis = 1), df['label']
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

svc=LinearSVC(dual=False)
scores = cross_val_score(svc, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

[0.82905983 0.82905983 0.84615385 0.88034188 0.8362069  0.73275862
 0.81896552 0.82758621 0.86206897 0.86206897]
0.8324270557029179


In [ ]:
C_range=list(range(1,26))
acc_score=[]
for c in C_range:
    svc = LinearSVC(dual=False, C=c)
    scores = cross_val_score(svc, X, y, cv=10, scoring='accuracy')
    acc_score.append(scores.mean())
    
C_values=list(range(1,26))
fig = go.Figure(data=go.Scatter(x=C_values, y=acc_score))
fig.update_layout(xaxis_title='Value of C for SVC',
                   yaxis_title='Cross Validated Accuracy', template='plotly_white',xaxis = dict(dtick = 1))
fig.show()


C_range=list(np.arange(0.1,6,0.1))
acc_score=[]
for c in C_range:
    svc = LinearSVC(dual=False, C=c)
    scores = cross_val_score(svc, X, y, cv=10, scoring='accuracy')
    acc_score.append(scores.mean())
    
C_values=list(np.arange(0.1,6,0.1))
fig = go.Figure(data=go.Scatter(x=C_values, y=acc_score))
fig.update_layout(xaxis_title='Value of C for SVC',
                   yaxis_title='Cross Validated Accuracy', template='plotly_white',xaxis = dict(dtick = 1))
fig.show()

In [ ]:
#bayes_native
from sklearn.naive_bayes import GaussianNB
X, y = df.drop(['title', 'text', 'source', 'label', 'text_standard'], axis = 1), df['label']
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
bayes_=GaussianNB()
scores = cross_val_score(bayes_, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())





[0.60683761 0.58974359 0.64957265 0.65811966 0.6637931  0.70689655
 0.65517241 0.62931034 0.64655172 0.68103448]
0.6487032124963159


In [ ]:
#SVM with poly kernel
from sklearn.svm import SVC
X, y = df.drop(['title', 'text', 'source', 'label', 'text_standard'], axis = 1), df['label']
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
SVC_=SVC(kernel="poly")
scores = cross_val_score(SVC_, X, y, cv=10, scoring='accuracy')
print(scores)
print(scores.mean())

C_range=list(range(1,26))
acc_score=[]
for c in C_range:
    SVC_=SVC(kernel="poly",C=c)
    scores = cross_val_score(SVC_, X, y, cv=10, scoring='accuracy')
    acc_score.append(scores.mean())
    
C_values=list(range(1,26))
fig = go.Figure(data=go.Scatter(x=C_values, y=acc_score))
fig.update_layout(xaxis_title='Value of C for svm with poly kernel',
                   yaxis_title='Cross Validated Accuracy', template='plotly_white',xaxis = dict(dtick = 1))
fig.show()


C_range=list(np.arange(0.1,6,0.1))
acc_score=[]
for c in C_range:
    SVC_=SVC(kernel="poly",C=c)
    scores = cross_val_score(SVC_, X, y, cv=10, scoring='accuracy')
    acc_score.append(scores.mean())
    
C_values=list(np.arange(0.1,6,0.1))
fig = go.Figure(data=go.Scatter(x=C_values, y=acc_score))
fig.update_layout(xaxis_title='Value of C for svm with poly kernel',
                   yaxis_title='Cross Validated Accuracy', template='plotly_white',xaxis = dict(dtick = 1))
fig.show()

[0.73504274 0.7008547  0.70940171 0.77777778 0.75862069 0.63793103
 0.74137931 0.68103448 0.75862069 0.75862069]
0.7259283819628648
